In [1]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from datasets import load_dataset
from scipy.special import softmax
import evaluate
import numpy as np

In [2]:
ds = load_dataset('imdb')

In [3]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [6]:
def preprocess_text(ds):
    return tokenizer(ds["text"], truncation=True)

In [7]:
tokenized_ds = ds.map(preprocess_text, batched=True)

In [8]:
roc_auc = evaluate.load('roc_auc', 'binary')
accuracy = evaluate.load('accuracy')
f1 = evaluate.load('f1')
precision = evaluate.load('precision')

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = np.argmax(logits, axis=1)
    probs = softmax(pred)
    return {
        'accuracy': accuracy.compute(predictions=pred, references=labels)['accuracy'],
        'f1': f1.compute(predictions=pred, references=labels)['f1'],
        'precision': precision.compute(predictions=pred, references=labels)['precision'],
        'roc_auc': roc_auc.compute(prediction_scores=probs, references=labels)['roc_auc']
    }

In [10]:
small_train = tokenized_ds["train"].shuffle().select(range(1000))
small_eval = tokenized_ds["test"].shuffle().select(range(1000))

In [11]:
training_args = TrainingArguments(
    output_dir="sentiment analysis model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [12]:
def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased",
    num_labels=2,
    id2label=id2label, 
    label2id=label2id
    )

In [13]:
trainer = Trainer(
    model=None,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_eval,
    processing_class=tokenizer,
    model_init=model_init,
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def compute_objective(metrics):
    combined_metric = metrics['eval_accuracy'] + metrics['eval_f1'] + metrics['eval_precision'] + metrics['eval_roc_auc']
    loss = metrics['eval_loss']
    return (combined_metric, loss)

In [15]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [32, 64, 128]),
        'num_train_epochs': trial.suggest_int('num_train_epochs', 1, 11, step=5)
    }

best_trials = trainer.hyperparameter_search(
    direction=['maximize', 'minimize'],
    backend="optuna",
    hp_space=optuna_hp_space,
    n_trials=20,
    compute_objective=compute_objective
)

[I 2025-04-24 03:17:07,693] A new study created in memory with name: no-name-f7178c36-f2b9-4f1a-955a-0f1ef2112b34
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Roc Auc
1,No log,0.689906,0.575000,0.468085,0.663121,0.582507
2,No log,0.682819,0.603000,0.435277,0.822581,0.613808
3,No log,0.672278,0.704000,0.655814,0.822157,0.709580
4,No log,0.655566,0.703000,0.642599,0.850318,0.709566
5,No log,0.629450,0.758000,0.742004,0.826603,0.760988
6,No log,0.594202,0.781000,0.778116,0.817021,0.782346
7,No log,0.563743,0.789000,0.784474,0.831169,0.790628
8,No log,0.538617,0.802000,0.797546,0.845987,0.803677
9,No log,0.518485,0.808000,0.806841,0.840671,0.809139
10,No log,0.507491,0.815000,0.813320,0.850211,0.816250


[I 2025-04-24 03:19:58,981] Trial 0 finished with values: [3.2982591473298313, 0.5038794875144958] and parameters: {'learning_rate': 2.2400641103816993e-06, 'per_device_train_batch_size': 32, 'num_train_epochs': 11}.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Roc Auc
1,No log,0.475457,0.807000,0.796199,0.876744,0.809738
2,No log,0.302014,0.881000,0.887417,0.868519,0.880079
3,No log,0.284385,0.894000,0.897485,0.897485,0.893877
4,No log,0.299801,0.898000,0.902857,0.889306,0.897337
5,No log,0.361596,0.897000,0.901435,0.892045,0.896506
6,No log,0.406447,0.891000,0.897844,0.870909,0.889751
7,No log,0.419184,0.896000,0.900383,0.891841,0.895539
8,No log,0.460134,0.890000,0.891945,0.906188,0.890417
9,No log,0.450889,0.894000,0.898855,0.887006,0.893401
10,No log,0.466307,0.898000,0.900391,0.909270,0.898222


[I 2025-04-24 03:32:15,500] Trial 1 finished with values: [3.58974838218208, 0.4759913384914398] and parameters: {'learning_rate': 3.807241430960454e-05, 'per_device_train_batch_size': 64, 'num_train_epochs': 11}.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

[I 2025-04-24 06:32:44,605] Trial 17 finished with values: [3.5835588279810042, 0.37692877650260925] and parameters: {'learning_rate': 1.947056541725573e-05, 'per_device_train_batch_size': 64, 'num_train_epochs': 11}.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Roc Auc
1,No log,0.694192,0.488000,0.375610,0.508251,0.494692
2,No log,0.692226,0.529000,0.330014,0.623656,0.539722
3,No log,0.690744,0.540000,0.311377,0.688742,0.551926
4,No log,0.689741,0.532000,0.254777,0.720721,0.545278
5,No log,0.689004,0.541000,0.281690,0.737705,0.553914
6,No log,0.688736,0.543000,0.284820,0.745902,0.555917


[I 2025-04-24 06:50:07,706] Trial 18 finished with values: [2.1296383102785854, 0.6887364983558655] and parameters: {'learning_rate': 2.069941910959875e-06, 'per_device_train_batch_size': 128, 'num_train_epochs': 6}.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Roc Auc
1,No log,0.694981,0.457000,0.394649,0.465789,0.461035
2,No log,0.693584,0.502000,0.353247,0.537549,0.510410
3,No log,0.692560,0.520000,0.333333,0.591133,0.530133
4,No log,0.691879,0.526000,0.286145,0.646259,0.538046
5,No log,0.691404,0.532000,0.293051,0.668966,0.544121
6,No log,0.691234,0.533000,0.293495,0.673611,0.545156


[I 2025-04-24 07:04:58,858] Trial 19 finished with values: [2.0452620166714035, 0.69123375415802] and parameters: {'learning_rate': 1.44356019589745e-06, 'per_device_train_batch_size': 128, 'num_train_epochs': 6}.


In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Roc Auc
1,No log,0.678391,0.630000,0.633663,0.592593,0.632878
2,No log,0.708419,0.640000,0.647059,0.600000,0.643517


{'accuracy': 0.63, 'f1': 0.6336633663366337, 'precision': 0.5925925925925926, 'roc_auc': 0.6328783621035728}
{'accuracy': 0.64, 'f1': 0.6470588235294118, 'precision': 0.6, 'roc_auc': 0.6435166599759132}


TrainOutput(global_step=14, training_loss=0.2708761351449149, metrics={'train_runtime': 5.248, 'train_samples_per_second': 38.11, 'train_steps_per_second': 2.668, 'total_flos': 25946016810192.0, 'train_loss': 0.2708761351449149, 'epoch': 2.0})

In [36]:
results

NameError: name 'results' is not defined

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision
1,No log,0.661977,0.630000,0.393443,0.857143
2,No log,0.650482,0.690000,0.607595,0.750000


logits [[-0.04660358 -0.23375049]
 [-0.13670678 -0.10802048]
 [ 0.04001261 -0.23190552]
 [ 0.05575487 -0.23637293]
 [-0.0457897  -0.15124926]
 [-0.05000955 -0.17469226]
 [-0.15288801 -0.11614197]
 [-0.01152636 -0.21166293]
 [-0.00869801 -0.15249668]
 [-0.08077078 -0.14809582]
 [-0.05842046 -0.17459695]
 [ 0.0862514  -0.28164783]
 [ 0.05259624 -0.25397056]
 [-0.0023636  -0.28595626]
 [-0.06346934 -0.11055364]
 [ 0.04421073 -0.18892325]
 [-0.04360927 -0.25736746]
 [ 0.03814324 -0.2788692 ]
 [ 0.01327825 -0.18881446]
 [-0.09355123 -0.17405675]
 [-0.03577552 -0.25836244]
 [ 0.05434755 -0.27062386]
 [ 0.01273063 -0.21328023]
 [-0.14728864 -0.13143681]
 [ 0.11737015 -0.30923447]
 [-0.17709984 -0.07700922]
 [-0.06231025 -0.21241571]
 [ 0.05190599 -0.27577394]
 [-0.10867372 -0.11169931]
 [-0.12813894 -0.07832247]
 [-0.19858006 -0.12412699]
 [-0.00064888 -0.27195218]
 [ 0.0068058  -0.17954671]
 [-0.0189494  -0.2326941 ]
 [-0.03986066 -0.13679364]
 [-0.04122387 -0.15880439]
 [-0.06984933 -0.2018

TrainOutput(global_step=14, training_loss=0.6314302171979632, metrics={'train_runtime': 13.1854, 'train_samples_per_second': 15.168, 'train_steps_per_second': 1.062, 'total_flos': 25946016810192.0, 'train_loss': 0.6314302171979632, 'epoch': 2.0})

In [49]:


# Fake example
pred = np.array([0, 1, 1, 0])
labels = np.array([0, 1, 0, 0])

result = accuracy.compute(predictions=pred, references=labels)
print(result)

{'accuracy': 0.75}
